In [ ]:
!pip install fastapi uvicorn pyngrok transformers --quiet


In [ ]:
from fastapi import FastAPI, UploadFile, File ,Request
from fastapi.responses import JSONResponse
from pydantic import BaseModel
import pandas as pd
from typing import List
from transformers import (
    AutoModelForTokenClassification, AutoTokenizer, TokenClassificationPipeline,
    AutoModelForSequenceClassification, TextClassificationPipeline
)
import torch
from pyngrok import ngrok
import uvicorn
import nest_asyncio
import json

nest_asyncio.apply()


In [ ]:
ngrok.set_auth_token("2z9U0xaAKf5QWl10rKPg2xVQEVo_6EcE5gViQAwkcbcdP73iS")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
aspect_model_path = "/content/drive/MyDrive/absa_model/aspect_extractor/"
sentiment_model_path = "/content/drive/MyDrive/absa_model/sentiment_classifier/"

# Load aspect extractor
aspect_model = AutoModelForTokenClassification.from_pretrained(aspect_model_path)
aspect_tokenizer = AutoTokenizer.from_pretrained(aspect_model_path)
aspect_pipeline = TokenClassificationPipeline(
    model=aspect_model,
    tokenizer=aspect_tokenizer,
    aggregation_strategy=None,
    device=0 if torch.cuda.is_available() else -1
)

# Load sentiment classifier
sentiment_model = AutoModelForSequenceClassification.from_pretrained(sentiment_model_path)
sentiment_tokenizer = AutoTokenizer.from_pretrained(sentiment_model_path)
sentiment_pipeline = TextClassificationPipeline(
    model=sentiment_model,
    tokenizer=sentiment_tokenizer,
    return_all_scores=False,
    device=0 if torch.cuda.is_available() else -1
)


Device set to use cpu
Device set to use cpu
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [ ]:
def extract_aspects(sentence: str):
    predictions = aspect_pipeline(sentence)
    aspects = []
    current_aspect = ""
    current_scores = []

    for pred in predictions:
        token = pred['word']
        label = pred['entity']
        score = pred['score']

        if token.startswith("##"):
            token = token[2:]
            current_aspect += token
            current_scores.append(score)
        else:
            if label == "B-ASP":
                if current_aspect:
                    avg_score = float(sum(current_scores) / len(current_scores))
                    aspects.append((current_aspect.strip(), avg_score))
                current_aspect = token
                current_scores = [score]
            elif label == "I-ASP":
                current_aspect += " " + token
                current_scores.append(score)
            else:
                if current_aspect:
                    avg_score = float(sum(current_scores) / len(current_scores))
                    aspects.append((current_aspect.strip(), avg_score))
                    current_aspect = ""
                    current_scores = []
    if current_aspect:
        avg_score = float(sum(current_scores) / len(current_scores))
        aspects.append((current_aspect.strip(), avg_score))
    return aspects

def predict_sentiment(sentence: str, aspects: list):
    results = []
    for aspect, asp_score in aspects:
        model_input = f"{aspect} </s> {sentence}"
        pred = sentiment_pipeline(model_input)[0]
        results.append({
            "aspect": aspect,
            "aspect_score": round(float(asp_score), 2),
            "sentiment": pred["label"],
            "confidence": round(float(pred["score"]), 2)
        })
    return results


In [ ]:
app = FastAPI()

class SentenceRequest(BaseModel):
    sentence: str

@app.post("/predict-text")
async def predict_text(data: SentenceRequest):
    aspects = extract_aspects(data.sentence)
    sentiments = predict_sentiment(data.sentence, aspects)
    return sentiments

@app.post("/predict-file")
async def predict_file(request: Request):
    data = await request.json()
    comments = data.get("comments")
    print(comments)
    if not comments or not isinstance(comments, list):
        return JSONResponse(status_code=400, content={"error": "Expecting JSON with 'comments' list."})

    all_results = []
    for sentence in comments:
        aspects = extract_aspects(sentence)
        sentiments = predict_sentiment(sentence, aspects)
        all_results.append({"sentence": sentence, "aspects": sentiments})
    return {"results": all_results}  # retourne dict avec "results" clé


In [ ]:
# Lancer le serveur en arrière-plan
public_url = ngrok.connect(8000)
print("🚀 L'API est disponible à l'adresse :", public_url)

uvicorn.run(app, host="0.0.0.0", port=8000)


🚀 L'API est disponible à l'adresse : NgrokTunnel: "https://fa33-34-169-230-61.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [243]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     196.75.164.13:0 - "POST /predict-text HTTP/1.1" 200 OK
['The food was excellent and the waiter was very attentive.', 'I loved the ambiance but the service was slow.', 'The desserts were delicious but the main course was disappointing.', 'The staff was rude and the place was noisy.', 'The location is perfect but parking is difficult.', 'The restaurant was clean and the tables were nicely arranged.', 'I had a great time, especially enjoying the live music.', 'The prices are reasonable but the portions are small.', 'The chef prepared the steak exactly how I like it.', 'The waiting time was too long and the hostess was unfriendly.', 'The drinks were refreshing and well prepared.', 'I did not like the salad; it was too salty.', 'The seafood platter was fresh and tasty.', 'The waiter forgot our order twice, which was frustrating.', 'The view from the terrace is breathtaking, perfect for dinner.']
INFO:     196.75.164.13:0 - "POST /predict-file HTTP/1.1" 200 OK


In [ ]:
ngrok.kill()